In [81]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from keras.layers import Input, Dense, Lambda, Flatten, Reshape
from keras.layers import Conv1D,UpSampling1D
from keras.models import Model
from keras import backend as K

n_features = 4
sequence_length = 1024
boat_csv = pd.read_csv("Data/Boat_nominal_data/Boat_sequences_mix.csv")
boat_csv = boat_csv.drop(columns=["Unnamed: 0", "M0C", "M1C", "Acceleration","Speed"])
scaler = StandardScaler()
normal_data = scaler.fit_transform(boat_csv)
print(normal_data.shape)

boat_val = pd.read_csv("Data/Boat_nominal_data/Boat_sequence_mix_val.csv")
boat_val = boat_val.drop(columns=["Unnamed: 0", "M0C", "M1C", "Acceleration","Speed"])
scaler = StandardScaler()
val_nom_data = scaler.fit_transform(boat_val)

def prepare_sequences(data, batch_size):
    samples = []
    for i in range(0,data.shape[0], batch_size):
        sample = data[i:i+batch_size]	
        samples.append(sample)
    sequences = np.array(samples)
    trainX = np.reshape(sequences, (len(sequences), batch_size, n_features))
    return trainX


def prepare_data():    
    trainX_nominal = prepare_sequences(normal_data,1024) 
    print(trainX_nominal.shape)
    
    valX_nominal = prepare_sequences(val_nom_data,1024)
    print(valX_nominal.shape)

    return trainX_nominal, valX_nominal

trainX_nominal, valX_nominal = prepare_data()
n_sequences = len(trainX_nominal)
trainX_nominal = np.reshape(trainX_nominal, [n_sequences, sequence_length, n_features , 1])
print(trainX_nominal.shape)


(307200, 4)
(300, 1024, 4)
(30, 1024, 4)
(300, 1024, 4, 1)


In [87]:
from keras import Sequential
from keras.layers import MaxPooling1D, RepeatVector, LSTM, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras_preprocessing.sequence import TimeseriesGenerator

input_shape = (1024, n_features)
latent_dim = 20
filters = 16
kernel_size = 2
use_mse = True   
load_weights = False

units = len(trainX_nominal)
inputs = Input(shape=(sequence_length,n_features,1), name='Enc_input')
x = inputs
for i in range(2):
    filters *= 2
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=2,
               activation='relu')(x)
shape = K.int_shape(x)
x = Flatten()(x)
x = Dense(16)(x)
embeddings = Dense(latent_dim, name="embeddings")(x)

encoder = Model(inputs, embeddings, name='encoder')
encoder.summary()

latent_inputs = Input(shape=(latent_dim,))
x = Dense(units=(shape[1]*shape[2]*shape[3]))(latent_inputs)
x = Reshape((shape[1], shape[2], shape[3]))(x)

for i in range(2):
    x = Conv2DTranspose(filters=filters,
                        kernel_size=kernel_size,
                        strides=2,
                        activation='relu')(x)

outputs = Conv2DTranspose(filters=1,
                          kernel_size=kernel_size,
                          padding='same')(x)
decoder = Model(latent_inputs, outputs)
decoder.summary()

outputs = decoder(encoder(inputs))    
vae = Model(inputs, outputs)
#vae.summary()
vae.compile(optimizer='rmsprop', loss='mse')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Enc_input (InputLayer)       (None, 1024, 4, 1)        0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 512, 2, 32)        160       
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 256, 1, 64)        8256      
_________________________________________________________________
flatten_32 (Flatten)         (None, 16384)             0         
_________________________________________________________________
dense_49 (Dense)             (None, 16)                262160    
_________________________________________________________________
embeddings (Dense)           (None, 20)                340       
Total params: 270,916
Trainable params: 270,916
Non-trainable params: 0
_________________________________________________________________
____

In [88]:

vae.fit(trainX_nominal,
        trainX_nominal,
        epochs=40,
        batch_size=128)

vae.save_weights('Conv2d_ae.h5')


Epoch 1/40


128/300 [===========>..................] - ETA: 9s - loss: 1.0005

256/300 [========================>.....] - ETA: 1s - loss: 1.0053

300/300 [==============================] - 9s 29ms/step - loss: 1.0030


Epoch 2/40


...

300/300 [==============================] - 3s 11ms/step - loss: 0.0346


Epoch 40/40


128/300 [===========>..................] - ETA: 1s - loss: 0.0234

256/300 [========================>.....] - ETA: 0s - loss: 0.0301

300/300 [==============================] - 4s 12ms/step - loss: 0.0291


In [90]:

def return_mask(num, labels):
    return np.squeeze(np.argwhere(labels == num))


labels = pd.read_csv("Data/Boat_nominal_data/Boat_mix_labels.csv")
labels = labels.drop(columns="Unnamed: 0")
labels = np.array(labels)
masks = [return_mask(num, labels)[:, 0] for num in range(0, 9)]
encodings = encoder.predict(trainX_nominal)

print(encodings.shape)
for mask in masks:
    plt.scatter(x=encodings[:, 0][mask], 
                y=encodings[:, 1][mask], alpha=0.5)
plt.title('Encodings')
plt.legend(labels=np.arange(0, 9))
plt.show()


(300, 20)


In [94]:
runs = []
for mask in masks:
    run_for_class = trainX_nominal[mask]
    print(run_for_class.shape)
    runs.append(run_for_class)

for j in range(9):    
    for i in runs[j]:
        run = np.reshape(i, (-1, sequence_length, n_features,1))
        rec = vae.predict(run)
        rec = np.reshape(rec, (len(trainX_nominal[0]), n_features))
        reconstruction_df = pd.DataFrame(rec, columns=boat_csv.columns)
        plt.plot(reconstruction_df["Lon"], reconstruction_df["Lat"])
        plt.show()
        break


(200, 1024, 4, 1)
(9, 1024, 4, 1)
(9, 1024, 4, 1)
(14, 1024, 4, 1)
(14, 1024, 4, 1)
(13, 1024, 4, 1)
(13, 1024, 4, 1)
(15, 1024, 4, 1)
(13, 1024, 4, 1)
